In [10]:
#if(!require("knitr")) install.packages("knitr")
if(!require("tree")) install.packages("tree")
if(!require("readr")) install.packages("readr")
#if(!require("knitr")) install.packages("knitr")
if(!require("caret")) install.packages("caret")
#if(!require("gridExtra")) install.packages("gridExtra")
if(!require("randomForest")) install.packages("randomForest")
if(!require("foreach")) install.packages("foreach")
if(!require("e1071")) install.packages("e1071")
if(!require("furrr")) install.packages("furrr")
if(!require("parallel")) install.packages("parellel")
if(!require("doParallel")) install.packages("doParallel")
if(!require("fastAdaboost")) install.packages("fastAdaboost")




Loading required package: randomForest

randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘randomForest’


The following object is masked from ‘package:dplyr’:

    combine


The following object is masked from ‘package:ggplot2’:

    margin


Loading required package: e1071

Loading required package: fastAdaboost

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘fastAdaboost’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [11]:
require("readr")
require("tidyverse")
#require("knitr")
require("tree")
require("caret")
#require("gridExtra")
require("furrr")
require("parallel")
require("doParallel")
require("fastAdaboost")


Loading required package: fastAdaboost



In [4]:
# data parsing
soldat <- read_csv("soldat.csv")
soldat$y <- factor(soldat$y,levels = c(-1,1),labels = c("insoluble","soluble"))
n <- nrow(soldat)
p <- ncol(soldat)

Parsed with column specification:
cols(
  .default = col_double()
)

See spec(...) for full column specifications.



In [0]:
cluster <- makeCluster(detectCores() - 1) # convention to leave 1 core for OS
registerDoParallel(cluster)

In [0]:
v <- cor(soldat %>% select(-y), use = "pairwise.complete.obs")
high_corr <- colnames(soldat)[findCorrelation(v, cutoff = 0.95)]
soldat <- soldat %>% select(-all_of(high_corr))

In [0]:
set.seed(1234)
inTest <- createDataPartition(soldat$y,p = 0.5, list = FALSE)[,1]
test <- soldat[inTest,]
training <- soldat[-inTest,]

In [0]:
control <- trainControl(method = "cv", 
                       number = 5,
                       classProbs = TRUE, 
                       allowParallel = TRUE)
metric <- "Accuracy"

adaboost_stump <- train(y ~. , data = training, 
                             method = "adaboost", 
                             trControl = control,
                             tuneGrid = expand.grid(nIter = 2000, method = "stump"),
                             metric = metric)
save(adaboost_stump,file="adaboost_stump.Rdata")

In [0]:
# varying the interaction depth
tunegrid <- expand.grid(nIter = 2000, method = c("stump", "4-node", "8-node", "16-node"))
bt_models <- train(y ~. , data = training, 
                             method = "adaboost", 
                             trControl = control,
                             tuneGrid = tunegrid,
                             metric = metric)
save(bt_models,file="bt_models.Rdata")

In [0]:
print(bt_models)
#1:51

Random Forest 

2816 samples
  46 predictor
   2 classes: 'insoluble', 'soluble' 

No pre-processing
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 2252, 2253, 2253, 2253, 2253 
Resampling results across tuning parameters:

  mtry  ROC        Sens       Spec     
   2    0.8597128  0.9118608  0.5837567
   7    0.8579401  0.8952616  0.6071388
  12    0.8565826  0.8872468  0.6061998
  17    0.8557083  0.8861056  0.6164934
  22    0.8542419  0.8878183  0.6192971
  27    0.8538724  0.8878215  0.6239788
  32    0.8524348  0.8843848  0.6202536
  37    0.8519535  0.8809546  0.6240007
  42    0.8509517  0.8855276  0.6193059

ROC was used to select the optimal model using the largest value.
The final value used for the model was mtry = 2.
